In [85]:
import tatqa_utils
import pandas as pd
import requests

import pandas as pd
import pyreadstat
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import utils
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import numpy as np
from progress.bar import Bar
%load_ext autoreload
%autoreload 2

from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI
from langchain_community.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path=".langchain.db"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [68]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')
#model_name = "codellama-7b-hf"

In [72]:
def infer(triton_url, prompt):

        payload = {
            "text_input": prompt,
            "max_tokens": 500,
            "temperature": 0.0,
            "stream": False
        }    

        ret = requests.post(
            triton_url,
            json=payload,
        )
        res = ret.json()
        #print(res)
        query_response = res["text_output"]
        return query_response
    
infer(f"http://172.22.214.120:28800/v2/models/{model_name}/generate", "HI")

'HI-TECH\nThe HI-TECH is a 1960s-era, 19-foot, 10-inch, 2-stroke, 2-cylinder, 100 horsepower, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc, 1000cc,'

In [98]:
prompt = """Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.
Question: What was the percentage change in Value added tax receivables, net, noncurrent in 2019 from 2018? 
 Table: [['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]
 """
infer(f"http://172.22.214.120:28800/v2/models/{model_name}/generate", prompt)

"Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.\nQuestion: What was the percentage change in Value added tax receivables, net, noncurrent in 2019 from 2018? \n Table: [['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n

In [77]:
import triton_utils
model_name = "codellama-7b-instruct-hf"
os.environ["TRITON_LLM_ENDPOINT"] =  f"http://172.22.214.120:28800/v2/models/{model_name}/generate"
llm = triton_utils.TritonLLM3()
llm.invoke("So?32")

'So?320000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [87]:
model_name = "codellama-7b-instruct-hf"
os.environ["TRITON_LLM_ENDPOINT"] =  f"http://172.22.214.120:28800/v2/models/{model_name}/generate"
llm = triton_utils.TritonLLM3()

def gen_code_infer(model_url, question, table):
    prompt = f"Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.\nQuestion: {question} \n Table: {table}"
    print(prompt)    
    res = infer(model_url, prompt)
    #res = llm.invoke(prompt)
    #print(res)#
    code = res.replace('```python','').replace('```','')
    code = code.replace('Answer:','')
    code = code.replace(prompt,'').strip()
    
    print("S", code, "@")
    return code


def gen_code(llm, question, table):
    prompt = f"Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.\nQuestion: {question} \n Table: {table}"
    print(prompt)    
    #res = infer(model_url, prompt)
    res = llm.invoke(prompt)
    #print(res)#
    code = res.replace('```python','').replace('```','')
    code = code.replace('Answer:','')
    code = code.replace(prompt,'').strip()
    
    print("S", code, "@")
    return code

def exec_code(code, table):  
    try: 
        loc = locals()   
        if not "run()" in code:
            exec(code + f"\nr = run({table})\n", globals(), loc)
        else: 
            exec(code + "\nr = run()\n", globals(), loc)
        return loc['r']
    except Exception as e:
            s = '[Error]'+ str(e)
            print(s)
            return (s,'')
#table = "[['', '', 'Years Ended September 30,', ''], ['', '2019', '2018', '2017'], ['Fixed Price', '$  1,452.4', '$  1,146.2', '$  1,036.9'], ['Other', '44.1', '56.7', '70.8'], ['Total sales', '$1,496.5', '$1,202.9', '$1,107.7']]"
#question = 'What is the change in Fixed Price in 2019 from 2018?'
table = "[['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]"
question = 'What was the percentage change in Value added tax receivables, net, noncurrent in 2019 from 2018?'
code = gen_code_infer(f"http://172.22.214.120:28800/v2/models/{model_name}/generate", question, table)
#code = gen_code(llm, question, table)
exec_code(code, table)

Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.
Question: What was the percentage change in Value added tax receivables, net, noncurrent in 2019 from 2018? 
 Table: [['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]
S def run():
    return (592 - 519) / 519 * 100, 2 @


(14.065510597302506, 2)

In [104]:
prompt = "Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code. Question: What was the percentage change in Value added tax receivables, net, noncurrent in 2019 from 2018?  Table: [['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]"
infer(f"http://172.22.214.120:28800/v2/models/{model_name}/generate", prompt)
#infer(f"http://172.22.214.120:28800/v2/models/{model_name}/generate", prompt)

"Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code. Question: What was the percentage change in Value added tax receivables, net, noncurrent in 2019 from 2018?  Table: [['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]\n\nAnswer:\n\n\\begin{code}\ndef run():\n    df = pd.read_csv('table.csv')\n    df = df.set_index(0)\n    df = df.iloc[2:]\n    df = df.drop(columns=[''])\n    df 

In [100]:
prompt = f"Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.\nQuestion: {question} \n Table: {table}"
infer(f"http://172.22.214.120:28800/v2/models/{model_name}/generate", prompt)


"Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.\nQuestion: What was the percentage change in Value added tax receivables, net, noncurrent in 2019 from 2018? \n Table: [['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]\n\nAnswer:\n\n```python\ndef run():\n    return (592 - 519) / 519 * 100, 2\n```\n"

In [89]:
gen_code_infer(f"http://172.22.214.120:28800/v2/models/{model_name}/generate", question, table)

Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.
Question: What was the percentage change in Value added tax receivables, net, noncurrent in 2019 from 2018? 
 Table: [['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]
S def run():
    return (592 - 519) / 519 * 100, 2 @


'def run():\n    return (592 - 519) / 519 * 100, 2'

In [81]:
from tqdm.notebook import tqdm as log_progress

res = []
for i, item in log_progress(devdf.iterrows()):
    table = item['table']['table']
    #print (i)
    for q in item['questions']:        
        if q['answer_type'] == 'arithmetic' and 'table' in q['answer_from'] : 
        #if True: 
            code = gen_code(llm, q, table)
            r = exec_code(code, table)
            err =  None
            if r is None:
                (pred_value, pred_unit) = (None, None)
            else:                
                if len(r) == 2:
                    (pred_value, pred_unit) = r                    
                    #print(pred_value, type(pred_value))
                    if isinstance(pred_value, str) and  pred_value.startswith('[Error]'):
                        (pred_value, pred_unit) = (None, None)
                        err,_ = r                                                        
                        
                elif len(r) == 1:
                    pred_value = r                
                else:
                    (pred_value, pred_unit) = (None, None)        
                
            #res.append({"table":table, "q":q, "pred":pred, "code": code})
            res.append(({"answer_type":q["answer_type"], "answer": q["answer"], 'scale': q["scale"]}, pred_value, pred_unit, q, code, item['table'], err))


0it [00:00, ?it/s]

Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.
Question: {'uid': 'eb787966-fa02-401f-bfaf-ccabf3828b23', 'order': 5, 'question': 'What is the change in Other in 2019 from 2018?', 'answer': -12.6, 'derivation': '44.1-56.7', 'answer_type': 'arithmetic', 'answer_from': 'table-text', 'rel_paragraphs': ['2'], 'req_comparison': False, 'scale': 'million'} 
 Table: [['', '', 'Years Ended September 30,', ''], ['', '2019', '2018', '2017'], ['Fixed Price', '$  1,452.4', '$  1,146.2', '$  1,036.9'], ['Other', '44.1', '56.7', '70.8'], ['Total sales', '$1,496.5', '$1,202.9', '$1,107.7']]
S  @
[Error]run() takes 0 positional arguments but 1 was given
Given the following table, can you generate a python code, without sample data, which can answer the follo

KeyboardInterrupt: 

In [9]:
from tatqa_metric import TaTQAEmAndF1

metrics = TaTQAEmAndF1()

for ans, pred, pred_scale, _,_, _,_ in res:
    metrics(ans, pred, pred_scale)
pred_em, pred_f1, scale_score, op_score = metrics.get_overall_metric(reset=True)
print( pred_em, pred_f1, scale_score)

special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
special case
0.6098514464425332 0.6340891321344798 0.6974198592650508


In [28]:
import pickle
with open('res_p8_unit.pickle', 'wb') as f:
    pickle.dump(res,f)

In [18]:
print([i for i in res if i[-1]  is not None][0])


({'answer_type': 'arithmetic', 'answer': 14.07, 'scale': 'percent'}, None, None, {'uid': 'a4f782f1-de3b-47a3-87af-1cb33c112b39', 'order': 6, 'question': 'What was the percentage change in Value added tax receivables, net, noncurrent in 2019 from 2018?', 'answer': 14.07, 'derivation': '(592-519)/519', 'answer_type': 'arithmetic', 'answer_from': 'table', 'rel_paragraphs': [], 'req_comparison': False, 'scale': 'percent'}, "def run():\n    table = [['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]\n    \n    value_2019 = int(table[5][1])\n    value_2018 = int(table[5][2])\n    \n    percentage_change = ((value_2019 - value_2

In [106]:
#jó
print( f"Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.\nQuestion: {question} \n Table: {table}")

Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.
Question: What was the percentage change in Value added tax receivables, net, noncurrent in 2019 from 2018? 
 Table: [['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]


In [107]:
#rossz ismétléses kódot ad
print("Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code. Question: What was the percentage change in Value added tax receivables, net, noncurrent in 2019 from 2018?  Table: [['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]")


Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code. Question: What was the percentage change in Value added tax receivables, net, noncurrent in 2019 from 2018?  Table: [['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]


In [108]:
## nm ad semmit
print( """Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.
Question: What was the percentage change in Value added tax receivables, net, noncurrent in 2019 from 2018? 
 Table: [['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]
 """)

Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.
Question: What was the percentage change in Value added tax receivables, net, noncurrent in 2019 from 2018? 
 Table: [['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]
 


In [109]:
infer(f"http://172.22.214.120:28800/v2/models/{model_name}/generate", f"Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.\nQuestion: {question} \n Table: {table}")

"Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.\nQuestion: What was the percentage change in Value added tax receivables, net, noncurrent in 2019 from 2018? \n Table: [['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]\n\nAnswer:\n\n```python\ndef run():\n    return (592 - 519) / 519 * 100, 2\n```\n"

In [110]:
infer(f"http://172.22.214.120:28800/v2/models/{model_name}/generate", "Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code. Question: What was the percentage change in Value added tax receivables, net, noncurrent in 2019 from 2018?  Table: [['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]")

"Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code. Question: What was the percentage change in Value added tax receivables, net, noncurrent in 2019 from 2018?  Table: [['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]\n\nAnswer:\n\n\\begin{code}\ndef run():\n    df = pd.read_csv('table.csv')\n    df = df.set_index(0)\n    df = df.iloc[2:]\n    df = df.drop(columns=[''])\n    df 

In [111]:
infer(f"http://172.22.214.120:28800/v2/models/{model_name}/generate", """Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.
Question: What was the percentage change in Value added tax receivables, net, noncurrent in 2019 from 2018? 
 Table: [['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]
 """)

"Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.\nQuestion: What was the percentage change in Value added tax receivables, net, noncurrent in 2019 from 2018? \n Table: [['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n

In [117]:
infer(f"http://172.22.214.120:28800/v2/models/{model_name}/generate", """Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.
Question: What was the percentage change in Value added tax receivables, net, noncurrent in 2019 from 2018? 
 Table: [['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]
 """)

"Given the following table, can you generate a python code, without sample data, which can answer the following question? the code must contain only one function called 'run', and no wrapping class. The function would return results with format (value, scale). Do not write explanation, just code.\nQuestion: What was the percentage change in Value added tax receivables, net, noncurrent in 2019 from 2018? \n Table: [['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n